In [ ]:
import numpy as np
import pickle
def load_mnist(normalize=True, flatten=True, one_hot_label=False):
  def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T): #60000개의 row를 하나씩 던져줌
      row[X[idx]] = 1 #label에 해당하는 값에 1 - 0000010000 이런식으로 만들어주기

    return T

  with open('/content/drive/MyDrive/mnist.pkl', 'rb') as f:
    dataset = pickle.load(f)

  if normalize: #이미지 읽어서 정규화
    for key in ('train_img', 'test_img'):
      dataset[key] = dataset[key].astype(np.float32)
      dataset[key] /= 255.0 #기존 데이터는 0-256까지 정수값 가지고 있음 ->0~1사이 값으로 표현

  if one_hot_label: #one-hot encoding
    dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
    dataset['test_label'] = _change_one_hot_label(dataset['test_label'])
    #60000개 숫자를 10개로 만들어줌
  
  if not flatten:
    for key in ('train_img', 'test_img'):
      dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

  return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label'])

In [ ]:
def cross_entropy_error(y,t):
 
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
    
  batch_size = y.shape[0]
  
  if y.size == t.size:
    t = t.argmax(axis=1)

  delta = 1e-7
  return -np.sum(np.log(y[np.arange(batch_size), t] + delta)) / batch_size

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
def softmax(x):
  exp_x = np.exp(x)
  sum_exp_x = sum(exp_x)
  y = exp_x / sum_exp_x
  
  return y

In [ ]:
def _numerical_gradient_no_batch(f,w): #여기서 x는 가중치!
  h = 1e-4
  grad = np.zeros_like(w)
  
  for idx in range(w.size):
    tmp_val = w[idx] #n번째 가중치

    #f(w+h)
    w[idx] = float(tmp_val) + h
    fxh1 = f(w)

    #f(w-h)
    w[idx] = float(tmp_val) - h
    fxh2 = f(w)

    grad[idx] = (fxh1 - fxh2) / (2 * h)
    w[idx] = tmp_val

  return grad

In [ ]:
def numerical_gradient(f,W): #여기서 X는 가중치(weight)!
  #샘플 1개인 경우는 그냥 호출
  if W.ndim == 1:
    return _numerical_gradient_no_batch(f,W)
  
  #샘플 여러개 한꺼번에 들어온 경우: batch_size = 2
  else:
    grad = np.zeros_like(W)
    
    for idx, W in enumerate(W):
      grad[idx] = _numerical_gradient_no_batch(f,W)

    return grad

In [ ]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    #파라미터 랜덤 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

  #forward 연산
  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    #활성화 함수
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    #logit에 softmax 함수 적용
    y = softmax(a2)

    return y

  #loss(교차 엔트로피) 구하기
  def loss(self, x, t):
    #x는 입력, t는 라벨
    y = self.predict(x)
    #y는 소프트맥스 통과한 값(예측값)
    return cross_entropy_error(y,t)

  #정확도 계산
  def accuracy(self, x, t):
    y = self.predict(x)
    #y, t는 원핫 형태 => argmax 통해 라벨 형식으로 바꿈
    y = np.argmax(y, axis=1) #batch 단위 학습 위해 axis=1
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0]) #x.shape[0]는 전체 개수
    return accuracy

  #각 파라미터의 기울기 구하기
  def numerical_gradient(self, x, t):
    #목적 함수 - cross entropy
    loss_W = lambda W : self.loss(x, t)

    grads = {}
    #목적함수에 대해 각 파라미터 별로 편미분
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

In [ ]:
#데이터 로딩 - 원핫 형식으로
(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, flatten=True, one_hot_label=True)

In [ ]:
y_train.shape

(60000, 10)

In [ ]:
#2층 신경망 객체 생성
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [ ]:
print(network.params['W1'].shape)
print(network.params['b1'].shape)
print(network.params['W2'].shape)
print(network.params['b2'].shape)

(784, 50)
(50,)
(50, 10)
(10,)


In [ ]:
x = np.random.rand(50, 784)
y = network.predict(x) #예측값
np.argmax(y[0])

8

In [ ]:
iters_num = 10000 #반복 횟수
train_size = x_train.shape[0] #훈련 데이터 크기
batch_size = 100 #미니배치 사이즈
learning_rate = 0.01 #학습률

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size, 1) #한 에폭당 반복 횟수. 최소 한번
#한 에폭은 전체 데이터 개수 60000개, 배치는 그걸 자른 것
#배치 사이즈 0될 수도 있으니까 1을 줘야 함

In [ ]:
#loss 점점 줄어드는 걸 보기 위함 - 시간 너무 오래 걸림(미분 계산량 많음)
for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  #랜덤하게 0 ~ train_size-1 안에 있는 숫자를 batch_size만큼 가져옴
  #index 반환. 중복 가능

  x_batch = x_train[batch_mask] #랜덤으로 배치사이즈만큼 훈련 데이터에서 선택
  y_batch = y_train[batch_mask] #랜덤으로 배치사이즈만큼 라벨에서 선택

  #각 파라미터의 gradient 계산
  grad = network.numerical_gradient(x_batch, y_batch)

  #각 파라미터를 업데이트 (ex. w = w - 학습률 * 기울기)
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

    #loss 구하기
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss) #loss 값을 train_loss_list에 추가

    #정확도 검사할 때는 batch가 아니라 전체 데이터 넣어줘야 함
    #파라미터 업데이트 후의 훈련 데이터 정확도
    train_acc = network.accuracy(x_train, y_train) #전체 데이터 accuracy
    #파라미터 업데이트 후의 테스트 데이터 정확도 - 안 본 데이터에 대한 정확도
    test_acc = network.accuracy(x_test, y_test)

    train_acc_list.append(train_acc) #accuracy 값을 acc_list에 추가
    test_acc_list.append(test_acc)
    
    #각 iter 마다: 파라미터 업데이트 후의 loss, train정확도, test 정확도 출력
    print(f"loss {loss}, train_accuracy {train_acc}, test_accuracy {test_acc}")

KeyboardInterrupt: ignored